In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from scipy.stats import norm
import pickle

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle

In [ ]:
from bayesee.evaluation import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
subject = "AZ"
file_name = repo_path / f"data/covert-search/large-field/p3_data_{subject}.pickle"

with open(file_name, "rb") as f:
    stimulus, response = pickle.load(f)

metadata = stimulus["metadata"]
spot_centers = metadata["spot_centers"]
monitor_width, monitor_height = metadata["monitor_size"]
stimulus_size = metadata["stimulus_size"]
n_location = metadata["n_location"]
spot_size = metadata["spot_size"]
stimulus_ppd = metadata["stimulus_ppd"]
target_amplitude = metadata["target_amplitude"]
target = metadata["target"]

file_name = (
    repo_path
    / f"data/covert-search/large-field/derived/p2_spatial_statistics_{subject}.csv"
)
spatial_statistics_human = pd.read_csv(file_name)
local_dp = spatial_statistics_human["dp"].values

location_near = np.arange(1, n_location)[
    spatial_statistics_human["ecc"] < spot_size * 1.5 / stimulus_ppd
]
n_location_near = len(location_near)

location_far = np.arange(1, n_location)[
    spatial_statistics_human["ecc"] >= spot_size * 1.5 / stimulus_ppd
]

In [ ]:
prior = np.array((0.5, *((0.5 / (n_location - 1),) * (n_location - 1))))
assert np.allclose(prior.sum(), 1.0)
log_prior_ratio = np.log(prior / prior[0])
log_likelihood_ratio = np.zeros_like(prior)

n_trial = 20000

target_location = np.zeros((n_trial,), dtype=np.int64)
target_location[: n_trial // 2] = np.random.randint(1, n_location, size=n_trial // 2)
response_location = np.zeros_like(target_location)

assert np.allclose(np.dot(target.flatten(), target.flatten()), 1.0)

In [ ]:
array_scale_dp = np.linspace(0, 6, 20)
model_simulation = pd.DataFrame()

for scale_dp in array_scale_dp:
    scale_local_dp = scale_dp * local_dp

    for index_trial in range(n_trial):
        array_standard_normal = np.random.normal(size=(n_location - 1,))
        log_likelihood_ratio[1:] = (
            array_standard_normal * scale_local_dp - scale_local_dp**2 / 2
        )
        if target_location[index_trial] > 0:
            log_likelihood_ratio[target_location[index_trial]] += (
                scale_local_dp[target_location[index_trial] - 1] ** 2
            )

        log_posterior_ratio = log_prior_ratio + log_likelihood_ratio
        response_location[index_trial] = np.argmax(log_posterior_ratio)

    model_simulation = pd.concat(
        [
            model_simulation,
            pd.DataFrame(
                {
                    "scale_dp": scale_dp,
                    "location": target_location,
                    "response_location": response_location,
                }
            ),
        ],
        ignore_index=True,
    )

In [ ]:
file_name = (
    repo_path
    / f"data/covert-search/large-field/derived/p3_scaled_ideal_observer_for_{subject}.csv"
)

model_simulation.to_csv(
    file_name,
    index=False,
)

In [ ]:
file_name = (
    repo_path
    / f"data/covert-search/large-field/derived/p3_scaled_ideal_observer_for_{subject}.csv"
)

model_simulation = pd.read_csv(file_name)
array_scale_dp = model_simulation["scale_dp"]
target_location = model_simulation["location"]
response_location = model_simulation["response_location"]
accurate_response = target_location == response_location

In [ ]:
unique_scale_dp = array_scale_dp.unique()
overall_accuracy = np.zeros_like(unique_scale_dp)
overall_cr_rate = np.zeros_like(unique_scale_dp)
overall_hit_rate = np.zeros_like(unique_scale_dp)
near_hit_rate = np.zeros_like(unique_scale_dp)
far_hit_rate = np.zeros_like(unique_scale_dp)

for scale_index, scale_dp in enumerate(unique_scale_dp):
    scale_condition = array_scale_dp == scale_dp
    overall_accuracy[scale_index] = accurate_response[scale_condition].mean()
    overall_cr_rate[scale_index] = accurate_response[
        scale_condition & (target_location == 0)
    ].mean()
    overall_hit_rate[scale_index] = accurate_response[
        scale_condition & (target_location != 0)
    ].mean()
    near_hit_rate[scale_index] = accurate_response[
        scale_condition & np.isin(target_location, location_near)
    ].mean()
    far_hit_rate[scale_index] = accurate_response[
        scale_condition & np.isin(target_location, location_far)
    ].mean()

In [ ]:
fig, ax = plt.subplots()
ax.plot(unique_scale_dp, overall_accuracy, label="overall_accuracy")
ax.plot(unique_scale_dp, overall_cr_rate, label="overall_cr_rate")
ax.plot(unique_scale_dp, overall_hit_rate, label="overall_hit_rate")
ax.plot(unique_scale_dp, near_hit_rate, label="near_hit_rate")
ax.plot(unique_scale_dp, far_hit_rate, label="far_hit_rate")

ax.legend(loc="best")
ax.set(xlim=(0, 6), xlabel="d' Scaling Factor", ylabel="Value")